# Análise de Grafos - Implementações em Python
Este notebook contém todas as funções utilizadas para análise de grafos, separadas por funcionalidades.

## Classe `Graph`

In [ ]:
class Graph:
    def __init__(self):
        self.vertices = [] 
        self.edges = []
        self.arcs = []

    def add_vertex(self, vertex):
        self.vertices.append(vertex)

    def add_edge(self, u, v, cost):
        self.edges.append((u, v, cost))

    def add_arc(self, u, v, cost):
        self.arcs.append((u, v, cost))

## Função de leitura de arquivo

In [ ]:
def read_file(path):
    try:
        with open(path, "r", encoding="utf-8") as arquivo:
            linhas = arquivo.readlines()
    except FileNotFoundError:
        print(f"Erro: Arquivo '{path}' não encontrado.")
        exit()
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")
        exit()

    vertices = set()
    arestas = set()
    arcos = set()
    vertices_requeridos = set()
    arestas_requeridas = set()
    arcos_requeridos = set()
    secao_atual = None

    for linha in linhas:
        linha = linha.strip()
        if not linha or linha.startswith("//") or linha.startswith("Name:"):
            continue
        if linha.startswith("ReN."):
            secao_atual = "ReN"
            continue
        elif linha.startswith("ReE."):
            secao_atual = "ReE"
            continue
        elif linha.startswith("EDGE"):
            secao_atual = "EDGE"
            continue
        elif linha.startswith("ReA."):
            secao_atual = "ReA"
            continue
        elif linha.startswith("ARC"):
            secao_atual = "ARC"
            continue

        if linha and secao_atual:
            partes = linha.split("\t")
            try:
                if secao_atual == "ReN":
                    vertice = int(partes[0].replace("N", ""))
                    demanda = int(partes[1])
                    custo_servico = int(partes[2])
                    vertices_requeridos.add((vertice, (demanda, custo_servico)))
                    vertices.add(vertice)
                elif secao_atual in ["ReE", "EDGE"]:
                    origem, destino = int(partes[1]), int(partes[2])
                    vertices.update([origem, destino])
                    aresta = (min(origem, destino), max(origem, destino))
                    custo_transporte = int(partes[3])
                    arestas.add((aresta, custo_transporte))
                if secao_atual == "ReE":
                    demanda = int(partes[4])
                    custo_servico = int(partes[5])
                    arestas_requeridas.add((aresta, (custo_transporte, demanda, custo_servico)))
                elif secao_atual in ["ReA", "ARC"]:
                    origem, destino = int(partes[1]), int(partes[2])
                    vertices.update([origem, destino])
                    arco = (origem, destino)
                    custo_transporte = int(partes[3])
                    arcos.add((arco, custo_transporte))
                    if secao_atual == "ReA":
                        demanda = int(partes[4])
                        custo_servico = int(partes[5])
                        arcos_requeridos.add((arco, (custo_transporte, demanda, custo_servico)))
            except (ValueError, IndexError):
                print(f"Erro ao processar linha: {linha}")
                continue

    if not vertices:
        print("Erro: Nenhum vértice encontrado no arquivo.")
        exit()

    return vertices, arestas, arcos, vertices_requeridos, arestas_requeridas, arcos_requeridos

## Validação do Grafo

In [ ]:
def validar_grafo(vertices, arestas, arcos):
    for (u, v), _ in arestas:
        if u not in vertices or v not in vertices:
            print(f"Erro: Aresta ({u}, {v}) contém vértices inexistentes.")
            exit()
    for (u, v), _ in arcos:
        if u not in vertices or v not in vertices:
            print(f"Erro: Arco ({u}, {v}) contém vértices inexistentes.")
            exit()
    print("Grafo validado com sucesso.")

## Cálculo de Graus

In [ ]:
def calcular_graus(vertices, arestas, arcos):
    graus = {v: [0, 0, 0] for v in vertices}
    for (u, v), _ in arestas:
        graus[u][0] += 1
        graus[v][0] += 1
    for (u, v), _ in arcos:
        graus[u][2] += 1
        graus[v][1] += 1
    return tuple((v, tuple(g)) for v, g in graus.items())

## Busca em Largura (BFS)

In [ ]:
def bfs(grafo, inicio):
    visitados = set()
    fila = [inicio]
    resultado = []
    while fila:
        vertice = fila.pop(0)
        if vertice not in visitados:
            visitados.add(vertice)
            resultado.append(vertice)
            fila.extend(v for v in grafo.get(vertice, []) if v not in visitados)
    return resultado

## Funções de Contagem

In [ ]:
def quantidade_vertices(grafo):
    return len(grafo)

def quantidade_arestas(grafo):
    return sum(len(vizinhos) for vizinhos in grafo.values()) // 2

def quantidade_arcos(grafo):
    arcos = 0
    for u in grafo:
        for v in grafo[u]:
            if u in grafo[v]:
                arcos += 1
    return arcos // 2

## Impressão de Graus Mínimo e Máximo

In [ ]:
def imprimir_graus(graus):
    grau_maximo = max(sum(g[1]) for g in graus)
    grau_minimo = min(sum(g[1]) for g in graus)
    print("Grau máximo:", grau_maximo)
    print("Grau mínimo:", grau_minimo)

## Algoritmo de Floyd-Warshall

In [ ]:
def floyd_warshall(vertices, edges, arcs):
    dist = {v: {u: float('inf') for u in vertices} for v in vertices}
    pred = {v: {u: None for u in vertices} for v in vertices}
    for v in vertices:
        dist[v][v] = 0
    for (u, v), cost in edges:
        dist[u][v] = cost
        dist[v][u] = cost
        pred[u][v] = u
        pred[v][u] = v
    for (u, v), cost in arcs:
        dist[u][v] = cost
        pred[u][v] = u
    for k in vertices:
        for i in vertices:
            for j in vertices:
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    pred[i][j] = pred[k][j]
    return dist, pred

## Cálculo do Caminho Médio

In [ ]:
def caminho_medio(vertices, edges, arcs):
    dist, _ = floyd_warshall(vertices, edges, arcs)
    soma = 0
    total_pares = 0
    for u in dist:
        for v in dist[u]:
            if u != v and dist[u][v] != float('inf'):
                soma += dist[u][v]
                total_pares += 1
    return soma / total_pares if total_pares > 0 else 0

## Cálculo da Densidade

In [ ]:
def calc_densidade(num_arestas, num_arcos, num_vertices):
    edges_max = num_vertices * (num_vertices - 1) / 2
    arcs_max = num_vertices * (num_vertices - 1)
    densidade = (num_arestas + num_arcos) / (edges_max + arcs_max)
    return densidade

## Cálculo do Diâmetro

In [ ]:
def calc_diametro(matriz_distancias):
    diametro = 0
    for origem in matriz_distancias:
        for destino in matriz_distancias[origem]:
            d = matriz_distancias[origem][destino]
            if d != float('inf') and origem != destino:
                diametro = max(diametro, d)
    return diametro

## Caminho Mínimo e Intermediação

In [ ]:
def caminho_minimo(matriz_pred, origem, destino):
    caminho = []
    atual = destino
    while atual is not None:
        caminho.insert(0, atual)
        if atual == origem:  
            break
        atual = matriz_pred[origem].get(atual)
    if caminho[0] != origem:
        return []
    return caminho

def criar_matriz_predecessores(vertices, arestas, arcos):
    _, predecessores = floyd_warshall(vertices, arestas, arcos)
    return predecessores

def calc_intermediacao(vertices, matriz_pred):
    intermediacao = {v: 0 for v in vertices}
    for origem in vertices:
        for destino in vertices:
            if origem != destino:
                caminho = caminho_minimo(matriz_pred, origem, destino)
                if len(caminho) > 1:
                    for v in caminho[1:-1]:
                        intermediacao[v] += 1
    return intermediacao

In [1]:

import pandas as pd

# Exemplo de como criar uma tabela com métricas de grafos
# Substitua pelos seus próprios dados ou integre com suas funções de cálculo
resultados = {
    "Grafo": ["grafo1", "grafo2"],
    "Número de vértices": [10, 8],
    "Número de arestas": [15, 12],
    "Densidade": [0.33, 0.43],
    "Caminho médio": [2.3, 1.8],
    "Diâmetro": [4, 3],
    "Intermediação (média)": [0.25, 0.31]
}

# Criando o DataFrame
df_resultados = pd.DataFrame(resultados)

# Exibindo a tabela no notebook
df_resultados.style.set_caption("Tabela de Métricas dos Grafos").set_properties(**{
    'background-color': '#f9f9f9',
    'color': '#333333',
    'border-color': '#cccccc',
    'border-width': '1px',
    'border-style': 'solid'
})


,Grafo,Número de vértices,Número de arestas,Densidade,Caminho médio,Diâmetro,Intermediação (média)
0,grafo1,10,15,0.330000,2.300000,4,0.250000
1,grafo2,8,12,0.430000,1.800000,3,0.310000
